#Anchor Backend GPU API

Purposes: Run inference on AI model, generate Manim animation gifs.

In [1]:
%%capture
!pip install opencv-python
!pip install pix2tex
!pip install fastapi
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install --upgrade google-cloud-storage

In [2]:
%%capture
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

In [29]:
import cv2
import numpy as np
import subprocess
import matplotlib.pyplot as plt
import nest_asyncio
from pyngrok import ngrok
import uvicorn
"""
Uses cv2 to find a red box 
Return a dictionary:
  {'box': [(top left point), (bottom right point)]
   'image': path to an isolated image of the equation
  }
"""
def get_eqn(img):
  img_hsv=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  # lower mask (0-10)
  lower_red = np.array([0,50,50])
  upper_red = np.array([10,255,255])
  mask0 = cv2.inRange(img_hsv, lower_red, upper_red)
  # upper mask (170-180)
  lower_red = np.array([170,50,50])
  upper_red = np.array([180,255,255])
  mask1 = cv2.inRange(img_hsv, lower_red, upper_red)
  # join my masks
  mask = mask0+mask1
  # set my output img to zero everywhere except my mask
  output_img = img.copy()
  output_img[np.where(mask==0)] = 0
  bool_image = mask.astype(bool)
  x, y = np.where(bool_image)  # get the indices (x, y) where the image is True
  # get top left and bottom right corners of red box
  map = {(x_i) + (y_i): (x_i, y_i) for x_i, y_i in zip(x, y)}
  bottom_right = max(map.keys())
  x_right, y_right = map[bottom_right]
  top_left = min(map.keys())
  x_left, y_left = map[top_left]

  box = img[x_left:x_right, y_left:y_right] # crop out box

  # remove any of the border we may have grabbed (keep only black pixels)
  for x in range(0, x_right-x_left):
    for y in range(0, y_right-y_left):
      if(max(box[x][y]) > 50):
        box[x][y] = (255,255,255)
  
  cv2.imwrite('output.jpeg', box)
  return {'box': [(x_left, y_left),(x_right, y_right)], 'image': 'output.jpeg'} # hardcoded now, will be multiple boxes later

# Run our fine tuned model on 
def image_to_latex(eqn_path):
  model_output = subprocess.check_output(['pix2tex', 'output.jpeg', '--checkpoint', '/content/mixed_e03_step16297 (2).pth']).decode("utf-8")
  print(model_output)
  parsed_model_output = model_output.split(':')[1].replace('\n', '').replace(' ', '').lower()
  return parsed_model_output

In [30]:
"""
Uses cv2 to extract the blue bordered region where we will put the graph
Returns [(top left point),(bottom right point)]
"""
def get_graph_box(img):
  img_hsv=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  lower_blue = np.array([101,50,38])
  upper_blue = np.array([110,255,255])  
  mask = cv2.inRange(img_hsv, lower_blue, upper_blue)
  output_img = img.copy()
  output_img[np.where(mask==0)] = 0
  bool_image = mask.astype(bool)
  x, y = np.where(bool_image)  # get the indices (x, y) where the image is True
  # get top left and bottom right corners of red box
  map = {(x_i) + (y_i): (x_i, y_i) for x_i, y_i in zip(x, y)}
  bottom_right = max(map.keys())
  x_right, y_right = map[bottom_right]
  top_left = min(map.keys())
  x_left, y_left = map[top_left]
  return [(x_left, y_left),(x_right, y_right)]

In [31]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_latex(image_path):
  img = cv2.imread(image_path)
  eqn_data = get_eqn(img)
  eqn_box = eqn_data['box']
  eqn_image = eqn_data['image']
  graph_box = get_graph_box(img)
  latex = image_to_latex(eqn_image)
  return {'eqn_box': eqn_box, 'latex': latex, 'graph_box': graph_box}

In [73]:
import re
import shutil
# returns file path
def generate_gif(graph_box, latex):

  width = graph_box[1][0] - graph_box[0][0]
  height = graph_box[1][1] - graph_box[1][0]
  print('Removing dirs')
  try:
    shutil.rmtree('/content/media/videos/compiled_manim_scene')
    print('Gone comp');
  except:
    pass
  
  try:
    shutil.rmtree('/content/media/videos/manim_scene')
    print('Gone normal')
  except:
    pass

  with open('manim_scene.py', 'r') as fin:

    s1 = re.sub(r"\'\$\$WIDTH\$\$\'", str(500), fin.read())
    s2 = re.sub(r"\'\$\$HEIGHT\$\$\'", str(500), s1)
    s3 = re.sub(r"\'\$\$TEX\$\$\'", "'" + latex + "'", s2)
    with open('compiled_manim_scene.py', 'w') as fout:
      fout.write(s3)
  """try:
    shutil.rmtree('/content/media/videos/compiled_manim_scene/')
  except:
    pass"""
  subprocess.run(['manim', 'compiled_manim_scene.py', 'Graph', '-r', '500,500', '-o', 'output.mp4', '--format=gif'])
  path = f'/content/media/videos/compiled_manim_scene/{500}p60/output.gif'
  
  return path


In [82]:
from google.cloud import storage


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [86]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import random
import uuid
import os
import base64
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="credentials.json" 

class Job(BaseModel):
    base64: str

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get("/")
def read_root():
    return {"Hello": "Welcome to the Anchor API"}


@app.post("/")
async def create_item(item: Job):
    input_path = 'input.jpeg'
    with open(input_path, "wb") as fh:
        fh.write(base64.b64decode(item.base64))
    data = get_latex('input.jpeg')  #return {'eqn_box': eqn_box, 'latex': latex, 'graph_box': graph_box}
    gif_path = generate_gif(data['graph_box'], data['latex'])

    data = get_latex('input.jpeg')  #return {'eqn_box': eqn_box, 'latex': latex, 'graph_box': graph_box}
    print(data)
    gif_path = generate_gif(data['graph_box'], data['latex'])
    gif_location = str(uuid.uuid4())
    img_location = str(uuid.uuid4())
    final_gif_path = upload_blob('anchorgifs', gif_path, gif_location)
    final_img_path = upload_blob('anchorgifs', input_path, img_location)
    result = {'box': [int(data['graph_box'][0][1]),int(data['graph_box'][0][0]), int(data['graph_box'][1][1]), int(data['graph_box'][1][0])], 'gif': f'https://anchorgifs.storage.googleapis.com/{gif_location}', 'img': f'https://anchorgifs.storage.googleapis.com/{img_location}'}
    print(result)
    return result

In [89]:
ngrok.set_auth_token('2FZC0hOsf6YkcyYGLy4A8814vCt_SuRuRgcURyhqi75whgQP')

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)